In [3]:
import pandas as pd

In [ ]:
PATH = "../data/"

In [ ]:
def standardize(filename, savename, id_alias, text_alias, construct_alias,
                labels_alias, labels_mapping,
                sep, construct = 'sentiment',
                counterfactual_included = False, read_data = False, data = None):
    
    # read file and return pandas dataframe with only id, text and construct label (column names in lower case)
    
    if not counterfactual_included and not read_data:
        data = pd.read_csv(filename, sep = sep)
    else:
        data = data
        
    data['text'] = data[text_alias]
    data[construct] = data[construct_alias]
    
    if id_alias:
        data['_id'] = data[id_alias]
    else:
        data['_id'] = range(len(data))
        
    if labels_mapping:
        data[construct] = data[construct].map(labels_mapping)
    
    
    data = data[['_id', 'text', construct]]
    
    print(data[construct].unique())
    data.to_csv(savename, sep = "\t", encoding = 'utf-8', index = False)
    

In [ ]:
# for all Kaushik data
train_type = ['train', 'test']
data_type = {'orig' : 'original', 'new': 'counterfactual'}

# for train in train_type:
#     for data in data_type.keys():
#         standardize(PATH+"counterfactual_data/sentiment/%s/%s.tsv" %(data, train),
#                    savename = PATH+"data/sentiment/%s/%s.csv" %(train, data_type[data]),
#                    id_alias = None,
#                    text_alias = 'Text',
#                    construct_alias = 'Sentiment',
#                    labels_alias = ['Positive', 'Negative'],
#                    labels_mapping = {'Positive' : 'positive', 'Negative': 'negative'},
#                    sep = "\t",
#                    construct = 'sentiment')


# standardize(PATH+"tweet-sentiment-extraction/test.csv",
#                    savename = PATH+"data/sentiment/test/kaggle.csv",
#                    id_alias = 'textID',
#                    text_alias = 'text',
#                    construct_alias = 'sentiment',
#                    labels_alias = ['positive', 'negative', 'neutral'],
#                    labels_mapping = None,
#                    sep = ",",
#                    construct = 'sentiment')

In [ ]:
# for sexism data 
# out-domain, aka EXIST misogyny data

data = pd.read_csv(PATH+"data/sexism/EXIST2021_dataset/training/EXIST2021_training.tsv", sep = "\t")
data = data[data['language'] == 'en']
data.head()


In [ ]:
# standardize(filename = PATH+"data/sexism/EXIST2021_dataset/training/EXIST2021_training.tsv",
#                    savename = PATH+"data/sexism/test/exist.csv",
#                    id_alias = 'id',
#                    text_alias = 'text',
#                    construct_alias = 'task1',
#                    labels_alias = ['sexist', 'non-sexist'],
#                    labels_mapping = None,
#                    sep = "\t",
#                    construct = 'sexism',
#                    read_data = True,
#                    data = data)

In [ ]:
# in-domain, aka the Samory data
data = pd.read_csv(PATH+"SAMORY_SEXIM.csv", sep = "\t")
original = data[data['of_id'].isna()]
counterfactual = data[~data['of_id'].isna()]

# choose one counterfactual for each original sexist example
counterfactual = counterfactual.groupby("of_id").apply(lambda x:x.sample(n=1)).reset_index(0, drop=True)

len(original), len(counterfactual), len(data)

In [ ]:
# what about train-test split? Maybe 70-30 for now
from sklearn.model_selection import train_test_split

train_original, test_original = train_test_split(original,
                                                    stratify=original['sexist'], 
                                                    test_size=0.3)
train_counterfactual, test_counterfactual = train_test_split(counterfactual,
                                                    stratify=counterfactual['sexist'], 
                                                    test_size=0.3)

In [ ]:
len(train_counterfactual), len(train_original)

In [ ]:
original.head()

In [ ]:
# dataset_names = ["train_original", "test_original", "train_counterfactual", "test_counterfactual"]

# for i, dataset in enumerate([train_original, test_original, train_counterfactual, test_counterfactual]):
#     standardize("",
#                    savename = PATH+"data/sexism/%s/%s.csv" %(dataset_names[i].split("_")[0],
#                                                              dataset_names[i].split("_")[1]),
#                    id_alias = '_id',
#                    text_alias = 'text',
#                    construct_alias = 'sexist',
#                    labels_alias = [True, False],
#                    labels_mapping = {True: 'sexist', False : 'non-sexist'},
#                    sep = "\t",
#                    construct = 'sexism',
#                    counterfactual_included = True,
#                    read_data = True,
#                    data = dataset)

In [ ]:
### for hate speech data
# standardize(filename, savename, id_alias, text_alias, construct_alias,
#                 labels_alias, labels_mapping,
#                 sep, construct = 'sentiment',
#                 counterfactual_included = False, read_data = False, data = None):

df = pd.read_csv("../data/data/hatespeech/VIDGEN_HATESPEECH.csv")

# refer to notebook 'explore data.ipynb' notebook in the DGHSD repo for details
counterfactuals = df[df['id'].str.endswith('p')]

# find their original counterpart
counterfactual_ids = [i[:-1] for i in counterfactuals['id'].values]
originals = df[df['id'].isin(counterfactual_ids)]

# leave the dev set alone for now
# for data in ['original_train', 'counterfactual_train', 'original_test', 'counterfactual_test']:
#     data_type, split = data.split("_")[0], data.split("_")[1]
    
#     if data_type == 'counterfactual':
#         data_slice = counterfactuals[counterfactuals['split'] == split]
#     else:
#         data_slice = originals[originals['split'] == split]
        
#     standardize(filename = None,
#                 savename = PATH+"data/hatespeech/%s/%s.csv" %(split, data_type),
#                 id_alias = 'id',
#                 text_alias = 'text',
#                 construct_alias = 'label',
#                 labels_alias = ['hate', 'nothate'],
#                 labels_mapping = {'hate': 'hate', 'nothate': 'not hate'},
#                 sep = ',',
#                 construct = 'hatespeech',
#                 counterfactual_included = False,
#                 read_data = True,
#                 data = data_slice)

In [ ]:
# out-of-domain data for hate speech: Hateval test set
# standardize(filename = PATH+"data/hatespeech/hateval2019/hateval2019_en_test.csv",
#                    savename = PATH+"data/hatespeech/test/hateval.csv",
#                    id_alias = 'id',
#                    text_alias = 'text',
#                    construct_alias = 'HS',
#                    labels_alias = [0, 1],
#                    labels_mapping = {0 :'not hate', 1: 'hate'},
#                    sep = ",",
#                    construct = 'hatespeech',
#                    read_data = False,
#                    data = data)

In [ ]:
def generate_and_store_adversarial_data(original_data, construct = 'sentiment', attack = 'inv', sample_size = 100):
    if attack == 'inv':
        from textattack.augmentation import CheckListAugmenter
        augmenter = CheckListAugmenter(pct_words_to_swap=0.2, transformations_per_example=1) #to-do: tweak params
        
        

    elif attack == 'swap':
        from textattack.augmentation import WordNetAugmenter
        augmenter = WordNetAugmenter(pct_words_to_swap=0.2, transformations_per_example=1) #to-do: tweak params
        
    new_data = []
    new_labels = []
    old_sents = []
        
    original_data = original_data.sample(n = sample_size)
    
    docs = list(original_data['text'])
    labels = list(original_data[construct])

    # Augment
    for n, doc in enumerate(docs):
        if n % 10 == 0:
            print(n, " done.")
        # todo: split multisentences, seperately augment them and then collate them back together
        from nltk.tokenize import sent_tokenize
        sents = sent_tokenize(doc)
        new_examples = []
        
        for sent in sents:
            new_examples.append(augmenter.augment(sent)[0])
        
        #print(new_examples)
        
        adversarial_doc = ' '.join(new_examples)
        new_data.append(adversarial_doc)
        new_labels.append(labels[n])
    
    print(len(new_data))
    print(len(docs))
    
    new_data = pd.DataFrame({"text": new_data, "original": docs, construct: new_labels})
    new_data = new_data[new_data['original'] != new_data['text']]
    new_data.to_csv(PATH+"data/%s/test/adv_%s.csv" %(construct, attack), sep = "\t")
    
    return new_data

In [ ]:
### generate adversararial data and store

attacks = [
           'adv_inv',
           'adv_swap']

# for construct in ['sentiment']:
#     for attack in attacks:
#         base = pd.read_csv(PATH+"data/%s/test/kaggle.csv" %(construct), sep = '\t')
#         generate_and_store_adversarial_data(base, attack = attack[4:], sample_size = 500)
        
# for construct in ['hatespeech', 'sexism', 'sentiment']:
#     for attack in attacks:
#         base = pd.read_csv(PATH+"data/%s/test/original.csv" %(construct), sep = '\t')
#         generate_and_store_adversarial_data(base, construct, attack = attack[4:], sample_size = min(800, len(base)))        

### Explore data

In [ ]:
training_data = {'sentiment': {'original' : 'sentiment/train/original.csv',\
                               'counterfactual' : 'sentiment/train/counterfactual.csv'
                              },
            'sexism': {'original' : 'sexism/train/original.csv',\
                               'counterfactual' : 'sexism/train/counterfactual.csv'
                              },
            'hatespeech' : {'original' : 'hatespeech/train/original.csv',\
                               'counterfactual' : 'hatespeech/train/counterfactual.csv'
                              },
                }

test_data = {'sentiment' : {'in-domain' : 'sentiment/test/original.csv',
                            'out-domain' : 'sentiment/test/kaggle.csv',
                            'adv_inv' : 'sentiment/test/adv_inv.csv',
                            'adv_swap' : 'sentiment/test/adv_swap.csv',
                            
                           },
             'sexism' : {'in-domain' : 'sexism/test/original.csv',
                            'out-domain' : 'sexism/test/exist.csv',
                            'adv_inv' : 'sexism/test/adv_inv.csv',
                            'adv_swap' : 'sexism/test/adv_swap.csv',                            
                           },
             'hatespeech' : {'in-domain' : 'hatespeech/test/original.csv',
                            'out-domain' : 'hatespeech/test/hateval.csv',
                            'adv_inv' : 'hatespeech/test/adv_inv.csv',
                            'adv_swap' : 'hatespeech/test/adv_swap.csv',                            
                           },
            }

labels = {'sentiment' : {'positive': 1, 'negative' : 0},
          'sexism' : {'sexist' : 1, 'non-sexist' : 0},
          'hatespeech' : {'hate' : 1, 'not hate' : 0}
                          }

In [ ]:
for construct in training_data.keys():
    print(construct)
    print()
    for datatype in training_data[construct]:   
        print('Training data')
        print()
        data = pd.read_csv(PATH+"data/" + training_data[construct][datatype], sep = "\t")
        data['%s_encoded' %construct] = data[construct].map( labels[construct])
        print(datatype)
        print()
        print(data.groupby("%s_encoded" %(construct)).size())
        print()
        print(data.groupby("%s" %(construct)).size())
        print()
    
    print("----------------------------------------------------------")
    print()
        
        

In [ ]:
for construct in training_data.keys():
    print(construct)
    print()

    for datatype in test_data[construct]:
        print('Test data')
        print()
        data = pd.read_csv(PATH+"data/" + test_data[construct][datatype], sep = "\t")
        print(datatype)
        print()
        print(data.groupby(construct).size())
        print()
    
    print("----------------------------------------------------------")
    print()
        
        

### Pair counterfactuals
Pair the counterfactuals in training data with their original counterpart to find diffs. The format should be:
original_id, original_text, original_label, counterfactual_text, counterfactual_label

In [ ]:
#fix label mapping for sentiment and hatespeech [fixed 23.04]
labels_mapping = {'hatespeech' : {'hate': 'hate', 'nothate': 'not hate'},
                  'sentiment' : {'Positive' : 'positive', 'Negative': 'negative'}
                 }

In [ ]:
paired_data = pd.DataFrame()
## sentiment: use only the train data specified by Kaushik et al

data = pd.read_csv("../data/counterfactual_data/sentiment/combined/paired/train_paired.tsv", sep = "\t")


originals = []
counterfactuals = []
original_labels = []
counterfactual_labels = []


for n in range(0, len(data), 2):
    originals.append(data['Text'].values[n])
    original_labels.append(data['Sentiment'].values[n])
    
for n in range(1, len(data), 2):
    counterfactuals.append(data['Text'].values[n])    
    counterfactual_labels.append(data['Sentiment'].values[n])

paired_data['original_text'] = originals
paired_data['counterfactual_text'] = counterfactuals
paired_data['original_label'] = original_labels
paired_data['counterfactual_label'] = counterfactual_labels
paired_data['original_id'] = [str(i) for i in range(0, len(paired_data))]
paired_data['counterfactual_id'] = [i+'p' for i in paired_data['original_id']]


paired_data['original_label'] = paired_data['original_label'].map(labels_mapping['sentiment'])
paired_data['counterfactual_label'] = paired_data['counterfactual_label'].map(labels_mapping['sentiment'])

paired_data.to_csv("../data/data/sentiment/train/paired.csv", sep = "\t", index = False)
paired_data

In [ ]:
paired_data = pd.DataFrame()


data = pd.read_csv("../data/data/sexism/SAMORY_SEXISM.csv", sep = "\t")
cf_data = pd.read_csv("../data/data/sexism/train/counterfactual.csv", sep = "\t")
original_data = pd.read_csv("../data/data/sexism/train/original.csv", sep = "\t")

originals = []
counterfactuals = []
original_labels = []
counterfactual_labels = []
original_ids = []

counterfactuals = cf_data['text']
counterfactual_labels = cf_data['sexism']
counterfactual_ids = cf_data['_id']

for n, row in cf_data.iterrows():
    cf_id = row['_id']
    og_id = data[data['_id'] == cf_id]['of_id'].values[0]
    og_tweet = data[data['_id'] == og_id]['text'].values[0]
    og_tweet_label = data[data['_id'] == og_id]['sexist'].values[0]
    
    original_ids.append(og_id)
    originals.append(og_tweet)
    original_labels.append('sexist') # as we only have counterfactuals for sexist examples
    
    
paired_data['original_id'] = original_ids
paired_data['counterfactual_id'] = counterfactual_ids
paired_data['original_text'] = originals
paired_data['counterfactual_text'] = counterfactuals
paired_data['original_label'] = original_labels
paired_data['counterfactual_label'] = counterfactual_labels

#paired_data.to_csv("../data/data/sexism/train/paired.csv", sep = "\t")
paired_data

In [ ]:
paired_data = pd.DataFrame()
## hate speech: use the training split specified by Vidgen et al

data = pd.read_csv("../data/data/hatespeech/VIDGEN_HATESPEECH.csv")

counterfactual_df = data[data['id'].str.endswith('p')].reset_index()
counterfactual_ids = [i[:-1] for i in counterfactual_df['id'].values]

originals = []
counterfactuals = []
original_labels = []
counterfactual_labels = []
original_ids = []
counterfactual_ids = []

counterfactuals = list(counterfactual_df['text'].values)
counterfactual_labels = list(counterfactual_df['label'].values)
counterfactual_ids = list(counterfactual_df['id'].values)

for n, row in counterfactual_df.iterrows():
    cf_id = row['id'][:-1]
    try:
        og_id = data[data['id'] == cf_id]['id'].values[0]
        og_tweet = data[data['id'] == og_id]['text'].values[0] 
        og_tweet_label = data[data['id'] == og_id]['label'].values[0]
    
        originals.append(og_tweet)
        original_labels.append(og_tweet_label) 
        original_ids.append(og_id)
    except: # apparently some counterfactuals do not have originals
        print(cf_id)
        
        # drop those from the cf list        
        
        del counterfactuals[n]
        del counterfactual_labels[n]
        del counterfactual_ids[n]
        
        pass
    

    
    
paired_data['original_id'] = original_ids
paired_data['counterfactual_id'] = counterfactual_ids
paired_data['original_text'] = originals
paired_data['counterfactual_text'] = counterfactuals
paired_data['original_label'] = original_labels
paired_data['counterfactual_label'] = counterfactual_labels

paired_data['original_label'] = paired_data['original_label'].map(labels_mapping['hatespeech'])
paired_data['counterfactual_label'] = paired_data['counterfactual_label'].map(labels_mapping['hatespeech'])

#paired_data.to_csv("../data/data/hatespeech/train/paired.csv", sep = "\t")
paired_data

### Delete data rows for EMNLP data submission

In [9]:
constructs = ['sentiment', 'sexism', 'hatespeech']
for construct in constructs:
    data = pd.read_csv("../data/data/%s/train/paired.csv" %(construct), sep = "\t")
#     data = data[['original_id', 'counterfactual_id',
#        'negation_additions', 'negation_deletions', 'affect word_additions',
#        'affect word_deletions', 'gender word_additions',
#        'gender word_deletions', 'identity word_additions',
#        'identity word_deletions', 'hedges_additions', 'hedges_deletions',
#        'hate words_additions', 'hate words_deletions']]
#     data.to_csv("../data/data/%s/train/paired.csv" %(construct), sep = "\t", index = False)
    print(data.head())

   original_id counterfactual_id  negation_additions  negation_deletions  \
0            0                0p               False               False   
1            1                1p               False               False   
2            2                2p               False                True   
3            3                3p               False                True   
4            4                4p               False               False   

   affect word_additions  affect word_deletions  gender word_additions  \
0                   True                   True                  False   
1                   True                   True                  False   
2                   True                   True                  False   
3                   True                   True                  False   
4                   True                   True                  False   

   gender word_deletions  identity word_additions  identity word_deletions  \
0                  F